# Project 2 Part 3A

**Student:** Matthew Malueg

## Imports, Loading, Preparing Files

In [18]:
# Import packages
import glob
import pandas as pd
import os, time, json
import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook
# Create the folder for saving files (if it doesn't exist)
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'movies.mwb',
 'movies.mwb.bak',
 'movies_erd.png',
 'movie_basics.csv',
 'movie_ratings.csv',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'tmdb_api_results2001.json',
 'tmdb_api_results2002.json']

In [11]:
# Load in the dataframe from Project 2 Part 1 as basics
basics = pd.read_csv('Data/movie_basics.csv')

**API Credentials**

In [12]:
with open('/Users/Malue/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
# Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [13]:
tmdb.API_KEY = login['api-key']

**Custom functions**

In [14]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)

    # Save the .info and .releases dictionaries
    info = movie.info()
    releases = movie.releases()

    # Loop through countries in releases
    for c in releases['countries']:
        # If the country abbreviation==US
        if c['iso_3166_1']=='US':
            # Save a 'certification' key in info with the fetched certification
            info['certification'] = c['certification']

    return info

In [15]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

**Test API credentials and fucntions**

In [6]:
avengers = get_movie_with_rating('tt0848228')
avengers

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 118.15,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path'

In [16]:
notebook = get_movie_with_rating('tt0332280')
notebook

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 61.889,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/mevhneWSqbjU22D1MXNd4H9x0r0.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

## Objectives - loading additional data for stakeholder

**Retrieve data with API calls to TMDB**

In [17]:
years = [2001, 2002]
errors = []

## OUTER LOOP
for YEAR in tqdm_notebook(years, f'Years completed'):
    
    ## CREATE NEW FILE FOR EACH YEAR
    # Defining the JSON file to store results for a certain year
    JSON_FILE = f'{FOLDER}tmdb_api_results{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If file does not exist: create it
    if file_exists == False:
        # Print message indicating so
        print(f'Creating {JSON_FILE} for API results for {YEAR}.')
        # Save empty dict with key 'imdb_id' and value 0
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}], f)
    
    elif file_exists == True:
        print(f'The file {JSON_FILE} already exists.')
    
    
    ## SELECT MOVIES FOR CURRENT {YEAR}
    # Saving new year as the current dataframe
    df = basics.loc[ basics['startYear'] == YEAR].copy()
    # Saving movie ids to separate variable
    movie_ids = df['tconst']
    
    # Load existing data from json into a dataframe called 'previous_df'
    previous_df = pd.read_json(JSON_FILE)
    
    # Filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    
    ## INNER LOOP
    for movie_id in tqdm_notebook(movie_ids_to_get, f'Movies from {YEAR}'): # (list, title of progress bar)
            
        # Get index and movie id from list
        try:
            # Retrieve the data for the movie_id
            temp = get_movie_with_rating(movie_id)
            # Append/extend results to existing file using a pre-made function
            write_json(temp, JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server with API calls
            time.sleep(0.02)
        except Exception as e:
            errors.append([movie_id, e])
    
    ## AFTER COMPLETING LOOP
    # Print a message showing the number of movie_id's that caused an error
    print(f'- Total errors: {len(errors)}')
    
    # Save file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f'{FOLDER}final_tmdb_data_{YEAR}.csv.gz', compression='gzip',
                         index=False)

Years completed:   0%|          | 0/2 [00:00<?, ?it/s]

Creating Data/tmdb_api_results2001.json for API results for 2001.


Movies from 2001:   0%|          | 0/1576 [00:00<?, ?it/s]

- Total errors: 219
Creating Data/tmdb_api_results2002.json for API results for 2002.


Movies from 2002:   0%|          | 0/1572 [00:00<?, ?it/s]

- Total errors: 497


**Combine API call results into single dataframe**

In [19]:
# Use glob to get all filepaths that match pattern
tmdb_files = sorted(glob.glob('Data/final_tmdb_data*.csv.gz'))
tmdb_files

['Data\\final_tmdb_data_2001.csv.gz', 'Data\\final_tmdb_data_2002.csv.gz']

In [21]:
# Use read_csv in a list comprehension and then combine with concat to load all files
df = pd.concat([pd.read_csv(f) for f in tmdb_files])
df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/tJLV3BAlHOgscVOrA99Wnb2gAef.jpg,NaN,48000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",NaN,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,"If they lived in the same century, they'd be p...",Kate & Leopold,0.0,6.319,1238.0,PG-13
2,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.000,3.0,NaN
3,tt0116916,0.0,/rFpHBidSlhjflmnLu7BZilyKeQR.jpg,NaN,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,73549.0,en,The Dark Mist,...,0.0,101.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,The Dark Mist,0.0,3.500,2.0,PG
4,tt0118589,0.0,/9NZAirJahVilTiDNCHLFcdkwkiy.jpg,NaN,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,10696.0,en,Glitter,...,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.405,132.0,PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,tt6449044,0.0,/a9pkw8stijESGx1flSGPqcXLkHu.jpg,"{'id': 957260, 'name': 'The Conman Collection'...",0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,314105.0,cn,賭俠2002,...,0.0,97.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,NaN,The Conman 2002,0.0,4.300,3.0,NaN
1291,tt6694126,0.0,/sXjVpTZyDvwzPVZve3AmyCUBeHk.jpg,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,819174.0,fa,عروس خوش‌قدم,...,0.0,101.0,"[{'english_name': 'Persian', 'iso_639_1': 'fa'...",Released,NaN,The Lucky Bride,0.0,8.000,1.0,NaN
1292,tt8302928,0.0,NaN,NaN,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",NaN,866533.0,el,Movie Toons: Treasure Island,...,0.0,0.0,[],Released,NaN,Movie Toons: Treasure Island,0.0,0.000,0.0,NaN
1293,tt8474326,0.0,NaN,NaN,0.0,[],NaN,292027.0,en,Skin Eating Jungle Vampires,...,0.0,0.0,[],Released,NaN,Skin Eating Jungle Vampires,0.0,0.000,0.0,NaN


In [23]:
df.to_csv('tmdb_results_combined.csv.gz', index=False)